In [1]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [2]:
filename = "../../data/04/ABox.ttl"

In [3]:
text1 = '''CQ_4.1
Return the assets that are part of the `house` asset and the asset percentage they represent, in descending order.
'''

query1 = '''
PREFIX : <https://w3id.org/sirius/ontology/data/04/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX hero: <https://w3id.org/sirius/ontology/hero/>

SELECT DISTINCT ?asset ?asset_part ?percentage
WHERE {
  ?asset a crm:E24_Physical_Human-Made_Thing ;
        crm:P46_is_composed_of ?asset_part .
  ?value_assessment hero:describes ?asset_part ;
                    hero:assessesPercentage ?percentage .
}
'''

In [4]:
text2 = '''CQ_4.2
Return the assets that are part of the `house` asset and the contributing values assigned to them, along with their score, dimension, aspect, note, documentation, and time interval.
'''

query2 = '''
PREFIX : <https://w3id.org/sirius/ontology/data/04/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX hero: <https://w3id.org/sirius/ontology/hero/>
PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
PREFIX tis: <http://ontologydesignpatterns.org/cp/owl/timeindexedsituation.owl#>

SELECT DISTINCT ?asset_part ?value ?score ?dimension ?aspect ?note ?document ?time_interval_start ?time_interval_end
WHERE {
    ?asset a crm:E24_Physical_Human-Made_Thing ;
            crm:P46_is_composed_of ?asset_part .
    ?value_assessment hero:describes ?asset_part ;
                        hero:assigns ?value ;
            hero:withinDimension ?dimension ;
            hero:withinAspect ?aspect ;
            hero:hasNote ?note ;
            hero:isDocumentedBy ?document ;
            tis:atTime ?time_interval .
    ?value hero:hasScore ?score .
    ?time_interval ti:hasIntervalStartDate ?time_interval_start ;
                    ti:hasIntervalEndDate ?time_interval_end .
}
'''

In [5]:
queries = [(text1, query1),
           (text2, query2)
           ]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_4.1
Return the assets that are part of the `house` asset and the asset percentage they represent, in descending order.

+-----------------------------------------------+---------------------------------------------------+--------------+
| asset                                         | asset_part                                        |   percentage |
|-----------------------------------------------+---------------------------------------------------+--------------|
| https://w3id.org/sirius/ontology/data/04/house | https://w3id.org/sirius/ontology/data/04/building  |           50 |
| https://w3id.org/sirius/ontology/data/04/house | https://w3id.org/sirius/ontology/data/04/furniture |           20 |
| https://w3id.org/sirius/ontology/data/04/house | https://w3id.org/sirius/ontology/data/04/mosaics   |           20 |
| https://w3id.org/sirius/ontology/data/04/house | https://w3id.org/sirius/ontology/data/04/paintings |           10 |
+-----------------------------------------------+-